In [5]:
#Instalando bibliotecas necessárioas

!pip install -qU langchain-groq
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [6]:
#Fazendo conexão com a API e pegando o modelo que será usado

import getpass
import os

os.environ["GROQ_API_KEY"] = "Token"

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant")

In [2]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory

In [51]:
#Esta é a função que vai guardar o chat na memória para que a IA lembre das mensagens

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

from langchain_core.runnables import Runnable

memory_store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in memory_store:
        memory_store[session_id] = ChatMessageHistory()
    return memory_store[session_id]

In [ ]:
#Fazendo um teste, dizendo para a IA quem sou eu e minha nacionalidade
#É possível ver que ele me respondeu com base no que eu disse

runnable_with_history = RunnableWithMessageHistory(
    runnable= llm,
    get_session_history=get_session_history,
)

runnable_with_history.invoke(
    [HumanMessage(content="hi - im Marcus Nunes. I'm Brazilian!")],
    config={"configurable": {"session_id": "1"}},
)

In [53]:
#É possível ver aqui também que ele lembra quem eu sou

runnable_with_history.invoke(
    [HumanMessage(content="hi - do you remember my name and where I come from?")],
    config={"configurable": {"session_id": "1"}},
)

AIMessage(content='Olá Marcus! Eu lembro que você é brasileiro. Você veio falar comigo apenas faz um momento atrás. Eu não tenho uma memória de longo prazo, então eu não posso lembrar de tudo que aconteceu anteriormente, mas eu lembro que você disse que é brasileiro e que se chama Marcus Nunes.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 115, 'total_tokens': 193, 'completion_time': 0.158356977, 'prompt_time': 0.006739061, 'queue_time': 0.213328931, 'total_time': 0.165096038}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8ab2e50475', 'finish_reason': 'stop', 'logprobs': None}, id='run--d49bbe4a-1c7d-40a7-8527-5b3584bf5779-0', usage_metadata={'input_tokens': 115, 'output_tokens': 78, 'total_tokens': 193})

In [58]:
#Aqui eu passo o promp template, onde eu posso configurar as respostas da minha IA, decidir o que ela vai falar e como vai falar

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are an assistat who speaks in {language}. Respond in 20 words or fewer", #Falei pra ela que ela é um assistente que fala em uma lingua que eu passar, e pra responder em 20 palavras ou menos
         ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm

runnable_with_history = RunnableWithMessageHistory(
    runnable= runnable,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)


In [59]:
runnable_with_history.invoke(
    {"language": "Portuguese", "input": "Hi im Marcus, I'm 21 years old!"}, # Aqui testo de novo a IA com o parâmetro de língua. Escolhi a lingua portuguesa
    config={"configurable": {"session_id": "2"}}, # Definindo uma nova sessão, a memória da IA reinicia e volta do zero
)

AIMessage(content='Olá Marcus! 21 anos, é uma idade incrível! (Hello Marcus! 21 years old, what an incredible age!)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 63, 'total_tokens': 92, 'completion_time': 0.051840697, 'prompt_time': 0.003706192, 'queue_time': 0.213876811, 'total_time': 0.055546889}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8ab2e50475', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0a753eb-e3a1-4370-87ae-0b25db63dd23-0', usage_metadata={'input_tokens': 63, 'output_tokens': 29, 'total_tokens': 92})

In [60]:
runnable_with_history.invoke(
    {"language": "Portuguese", "input": "What was my name and my age?"},
    config={"configurable": {"session_id": "2"}},
)

#Aqui é possível ver que a IA lembrou de novo quem eu era graças a função de armazenamento na memória

AIMessage(content='Seu nome é Marcus e você tem 21 anos.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 109, 'total_tokens': 122, 'completion_time': 0.017333333, 'prompt_time': 0.006408381, 'queue_time': 0.214515321, 'total_time': 0.023741714}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'finish_reason': 'stop', 'logprobs': None}, id='run--47252af3-0e91-4bc7-afdc-7cb04d24dff5-0', usage_metadata={'input_tokens': 109, 'output_tokens': 13, 'total_tokens': 122})

In [61]:
#Aqui testo o capacidade de resumo da IA passadno a história dos três porquinhos. Ao mesmo tempo que testo essa habilidade, eu testo a memória também na célula abaixo dessa

runnable_with_history.invoke(
    {"language": "Portuguese", "input": """Era uma vez três porquinhos que decidiram construir suas casas para se proteger do lobo mau. O primeiro porquinho fez sua casa de palha, porque era rápido e fácil. O segundo construiu a casa de madeira, um pouco mais forte. O terceiro, mais cuidadoso, fez sua casa de tijolos, bem resistente.

Um dia, o lobo chegou e soprou a casa de palha, que caiu facilmente. O porquinho correu para a casa de madeira. O lobo soprou com força, e a casa de madeira também desabou. Os dois porquinhos correram para a casa de tijolos.

O lobo tentou soprar, soprar, mas a casa de tijolos não caiu. Cansado, ele desistiu. Os três porquinhos aprenderam que, com esforço e cuidado, a proteção é mais forte.

Fim."""},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='Que história agradável! É um ensinamento valioso para todos. Proteção é mais forte com esforço e cuidado.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 352, 'total_tokens': 382, 'completion_time': 0.070467278, 'prompt_time': 0.020841706, 'queue_time': 0.217917103, 'total_time': 0.091308984}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8ab2e50475', 'finish_reason': 'stop', 'logprobs': None}, id='run--91a3b7fd-41e3-4972-a485-32e3affff1df-0', usage_metadata={'input_tokens': 352, 'output_tokens': 30, 'total_tokens': 382})

In [62]:
runnable_with_history.invoke(
    {"language": "Portuguese", "input": "faça um resumo de 1 linha da história que te contei"},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='Três porquinhos construiram casas com diferentes materiais, e a casa de tijolos foi a única a resistir ao ataque do lobo mau.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 406, 'total_tokens': 445, 'completion_time': 0.061671784, 'prompt_time': 0.024149708, 'queue_time': 0.21627752, 'total_time': 0.085821492}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'finish_reason': 'stop', 'logprobs': None}, id='run--57ea9481-a52d-4cfb-a8cc-1e029601d168-0', usage_metadata={'input_tokens': 406, 'output_tokens': 39, 'total_tokens': 445})